In [ ]:
import torch

In [ ]:
def int2bytes(tensor, bits=2):
    bool_tensors = [torch.zeros_like(tensor, dtype=torch.bool) for _ in range(bits)]     # construct a list of boolean tensor
    for i, bit in enumerate(reversed(range(bits))):
        idx = tensor >= (2**bit)
        bool_tensors[i][idx] = True
        tensor = tensor - (2**bit) * idx
    return bool_tensors

In [ ]:
def bytes2int(bool_tensors):
    tensor = torch.zeros_like(bool_tensors[0], dtype=torch.int)
    for i, bt in enumerate(reversed(bool_tensors)):
        tensor += (2 ** i) * bt
    return tensor

In [ ]:
a = torch.randint(4, (5, 6))
print(a)

In [ ]:
bool_a = int2bytes(a, bits=2)
for ba in bool_a:
    print(ba)

In [ ]:
re_a = bytes2int(bool_a)
print(re_a)

In [ ]:
# more complicated test
b = torch.randint(16, (10, 20, 30))
re_b = bytes2int(int2bytes(b, bits=4))
print(torch.sum(torch.abs(b-re_b)))